# Datenbeschaffung Wädenswil

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from locale import setlocale, LC_TIME
import os
import re
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Wir starten den Browser
driver = webdriver.Firefox()

# Wir sagen dem Browser, welche Seite er besuchen sollte.
driver.get('https://www.waedenswil.ch/politbusiness?politische_geschaefte_suchformular%5Bkeyword%5D=&politische_geschaefte_suchformular%5Bnummer%5D=&politische_geschaefte_suchformular%5BvomStart%5D=1.1.2000&politische_geschaefte_suchformular%5BvomEnd%5D=1.1.2024&politische_geschaefte_suchformular%5BstatusId%5D=&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=anfrage&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=dringliche_interpellation&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=einzelinitiative&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=erklaerung&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=interpellation&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=kleine_anfrage&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=motion&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=petition&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=postulat&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=volksinitiative&politische_geschaefte_suchformular%5BsearchBtn%5D=&politische_geschaefte_suchformular%5B_token%5D=-LtVJxTP2UgxWDiJ-lelrrHUs5y8lSRjhXGqmJd9gYc#')
#Die Hauptseite lädt relativ schnell, 2 Sekunden sollten reichen.
time.sleep(2)
#Es wird später das Problem auftauchen, dass die Elemente auf der Seite beim Laden in unterschiedlicher 
#Reihenfolge angezeigt werden. So werden Elemente mehrmals und andere gar nicht gespeichert. Ich sortiere also
#nach Datum, um das Problem zu beheben.
driver.find_element(By.CLASS_NAME, 'sorting').click()


# Nun gebe ich den Befehl, auf jede der 26 Seiten zuzugreifen.
for page_number in range(26):
    #Als nächstes muss ich auf die 20 Subpages zugreifen, die jeweils auf einer der 26 Seiten sind.
    for link_number in range(20):
        #Den Namen der Elemente suche ich über das Entwicklertool
        elements = driver.find_elements(By.XPATH,'//td[@class="icms-datatable-col-title"]/a')
        
    #Es gibt auf der Seite noch ein Weiteres Element mit dem gesuchten Namen. Das soll er ignorieren, 
    #deshalb die Try-Funktion.
        try:
            elements[link_number].click()
            time.sleep(6)
            #Ich speichere die Subpages in einem Ordner als htm Datei ab
            subpage = driver.page_source.encode('utf-8')
            with open("subpages_waedenswil/"+str(page_number)+'_'+str(link_number)+".htm", "wb+") as file:
                file.write(subpage)

            #Gehe zurück zur vorherigen Seite
            driver.back()
        #Hier überspringe ich das falsche Element.
        except Exception as e:
            print(f"Error: {e}")
            continue
        #Es dauert auch hier etwas
        time.sleep(3)

    #Dann soll das Programm zu nächsten Seite gehen. und in der Schlaufe oben wieder beginnen. Die Webseite
    #weist die aktuelle Seitenzahl leider nicht in der URL aus. Ich muss die Seite also mit einem Selenium-Befehl
    #wechseln...
    driver.find_element(By.ID, 'icmsTable-geschaefteList_next').click()
    time.sleep(4)

#Am Ende soll das Programm den Browser schliessen
driver.quit()

Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range
Error: list index out of range


Die htm-Files sind gespeichert. Nun werden sie in ein CSV zusammengefasst.

In [4]:
#Ich definiere, wo die Dokumente zu finden sind und schaue, wie viele es sind.
lst = os.listdir("subpages_waedenswil/")
len(lst)

507

In [5]:
#Ich sehe, dass es mir mehr Elemente anzeigt, als im Ordner sind... Ich schau mir das mal an.
for elem in tqdm(lst):
    print(elem)

100%|█████████████████████████████████████| 507/507 [00:00<00:00, 248076.54it/s]

20_10.htm
3_13.htm
13_2.htm
18_14.htm
11_18.htm
24_9.htm
16_11.htm
11_0.htm
19_9.htm
17_6.htm
14_14.htm
7_9.htm
22_15.htm
1_16.htm
9_6.htm
15_4.htm
15_5.htm
9_7.htm
1_17.htm
22_14.htm
7_8.htm
14_15.htm
17_7.htm
19_8.htm
11_19.htm
11_1.htm
16_10.htm
24_8.htm
18_15.htm
13_3.htm
3_12.htm
20_11.htm
20_13.htm
3_10.htm
4_19.htm
13_1.htm
18_17.htm
16_12.htm
11_3.htm
5_8.htm
14_17.htm
17_5.htm
22_16.htm
9_5.htm
1_15.htm
8_19.htm
15_7.htm
15_6.htm
8_18.htm
1_14.htm
9_4.htm
22_17.htm
17_4.htm
14_16.htm
5_9.htm
11_2.htm
16_13.htm
18_16.htm
13_0.htm
3_11.htm
4_18.htm
20_12.htm
20_16.htm
1_9.htm
13_4.htm
3_15.htm
16_17.htm
11_6.htm
18_12.htm
14_12.htm
22_9.htm
17_0.htm
22_13.htm
15_2.htm
9_0.htm
1_10.htm
6_19.htm
1_11.htm
6_18.htm
9_1.htm
15_3.htm
22_12.htm
17_1.htm
22_8.htm
14_13.htm
18_13.htm
11_7.htm
16_16.htm
3_14.htm
13_5.htm
1_8.htm
20_17.htm
20_15.htm
13_7.htm
3_16.htm
3_8.htm
16_14.htm
11_5.htm
18_11.htm
17_3.htm
13_18.htm
14_11.htm
22_10.htm
15_1.htm
20_8.htm
1_13.htm
9_3.htm
9_2.htm
1_12.

In [6]:
#Als erstes erstelle ich einen Platzhalter für meinen Datensatz
datensatz_waedenswil = []
#Es gibt ein verstecktes Element namens .DS_Store. Später wird es eine Fehlermeldung geben, weil er dessen
#Format nicht lesen kann. Ich mache sage also, dass er dieses File ignorieren soll.
for elem in tqdm(lst):
    if not elem.startswith('.DS_Store'):
        #Nun öffne ich eine Daten nach der anderen und lese sie ein.
        with open(os.path.join('subpages_waedenswil', elem), 'r') as file:
            text = file.read()
            #Ich lasse das Ganze mit BeautifulSoup lesen.
            soup = BeautifulSoup(text, 'html.parser')
            #Und definiere alle Elemente, die ich haben will.
            try:
                titel = soup.find(class_='contentTitle').text.strip()
            except:
                titel=None
            try:    
                geschaeftsart = soup.find(class_='icms-desclist-container').find('dt', string='Geschäftsart').find_next_sibling('dd').text.strip()
            except:
                geschaeftsart=None
            try:    
                status = soup.find(class_='icms-desclist-container').find('dt', string='Status').find_next_sibling('dd').text.strip()
            except:
                status=None
            try:
                datum = soup.find(class_='icms-desclist-container').find('dt', string='Datum').find_next_sibling('dd').text.strip()
            except:
                datum=None
            try:
                namen = soup.find(class_='icms-desclist-container').find('dt', string='Verfasser/Beteiligte').find_next_sibling('dd').findChildren()
                namen = [name.get_text(separator="\n") for name in namen]
            except:
                namen=None

            # Danach speichere ich diese Elemente in einem Dictionairy....
            mini_dict = {
                'Titel': titel,
                'Geschaeftsart': geschaeftsart,
                'Status': status,
                'Datum': datum,
                'Namen': namen
            }
            # ...und füge das Mini-Dictionary zur oben erstellten Liste hinzu.
            datensatz_waedenswil.append(mini_dict)

# Am Schluss mache ich daraus ein Dataframe mit Pandas.
df = pd.DataFrame(datensatz_waedenswil)

100%|█████████████████████████████████████████| 507/507 [00:06<00:00, 75.80it/s]


In [7]:
df

,Titel,Geschaeftsart,Status,Datum,Namen
0,Interpellation betreffend Ausbau Bahnhof Wädenswil,Interpellation,Erledigt,2. April 2018,"[ Reiter Ulrich, , Rita Hug (Erstunterzeichner/in)\n Claudia Bühlmann (Mitunterzeichner/in), ]"
1,Postulat betreffend Berücksichtigung der IAO-Kernübereinkommen im städtischen Beschaffungswesen,Postulat,Beantwortet,19. Juni 2008,[Willy Rüegg]
2,Interpellation betreffend Konzept für den Erhalt der Bienenvölker,Interpellation,Erledigt,17. Dezember 2012,"[ Baer Charlotte, , Jürg Wuhrmann (keine Funktion)\n Ivano Coduri (keine Funktion)\n Erich Schärer (keine Funktion)\n Marcel Bättig (keine Funktion)\n Christoph Lehmann (keine Funktion)\n Adrian Stocker (keine Funktion)\n Pascal Rubin (keine Funktion)\n Christina Zurfluh Fraefel (keine Funktion)\n Hanspeter Andreoli (keine Funktion)\n Sandy Bossert (keine Funktion), , , , , , , , , ]"
3,Postulat betreffend Unterstützung zur Senkung von Mietzinsen bei Sozialhilfebeziehenden,Postulat,Abgeschrieben,20. Juni 2017,"[ Reiter Ulrich, , Rita Hug (Erstunterzeichner/in)\n Claudia Bühlmann (Mitunterzeichner/in), ]"
4,Interpellation betreffend die nachhaltige Wasserversorgung in Wädenswil,Interpellation,Beantwortet,15. Juli 2012,[Albert A. Stahel (Verfasser/in)]
5,Schriftliche Anfrage betreffend Energieholz für Heizverbund-Anlagen,Anfrage,Beantwortet,16. Oktober 2022,"[ Fuchs Werner, Hauser Urs, Minutella Angelo, Reust Patrick, Roth Hans, Tanner Imfeld Monika, Willi Daniel, , Karin Signer (Mitunterzeichner/in)\n Judith Fürst (Mitunterzeichner/in)\n Edith Höhn (Erstunterzeichner/in)\n Gabi Bachmann (Mitunterzeichner/in), , , ]"
6,Interpellation betreffend Baurechtsverträge in Wädenswil,Interpellation,Erledigt,22. Juni 2015,"[Thomas Rom (keine Funktion)\n Peter Hildebrand (keine Funktion)\n Beatrice Gmür (Erstunterzeichner/in)\n Hansjörg Schmid (Mitunterzeichner/in)\n Edith Brunner (Mitunterzeichner/in)\n Christian Gross (Mitunterzeichner/in), , , , , ]"
7,Interpellation betreffend Parkplätze neue Stadtbibliothek,Interpellation,Beantwortet,30. Januar 2012,"[ Furrer Astrid, , Peter Dolder, Gian-Fadri Gattiker, Thomas Bürge\n Walter Münch (keine Funktion), ]"
8,Schriftliche Anfrage betreffend Einsatz von kombinierten Abfalleimern mit Aschenbechern,Anfrage,Beantwortet,15. November 2017,"[ Koch Thomas, Schafflützel Roman, Schüpbach Nadia , , Walter Münch (Mitunterzeichner/in)\n Ernst Grand (Mitunterzeichner/in)\n Christian Nufer (Mitunterzeichner/in), , ]"
9,Schriftliche Anfrage betreffend Öffentlichkeitsarbeit zur geplanten Gemeindefusion,Anfrage,Beantwortet,26. März 2016,"[ Roth Hans, , Peter Hildebrand (keine Funktion)\n Beatrice Gmür (Mitunterzeichner/in)\n Hansjörg Schmid (Mitunterzeichner/in)\n Edith Brunner (Erstunterzeichner/in)\n Christian Gross (Mitunterzeichner/in), , , , ]"


In [8]:
#Ich muss die Namensliste bereinigen. Dafür schreibe ich eine Funktion. In einigen Elementen trennt es die 
#Namen gar nicht, bei anderen Elementen hängtes Zusätze wie "Mitunterzeichner/in" oder eine Partei an.
def namenliste_reinigen(liste):
    
    if liste is None:
        return liste
    
    else:
        neueliste = []
        
        for element in liste:
            neueselement = element.strip()
            neueselement = neueselement.replace('Peter Dolder, Thomas Bürge', 'Peter Dolder\n Thomas Bürge').replace('Peter Dolder, Gian-Fadri Gattiker, Thomas Bürge', 'Peter Dolder\n Gian-Fadri Gattiker\n Thomas Bürge').replace('Corina Bürgi, Migmar Dhakyel, Willy Rüegg, Thomas Hartmann','Corina Bürgi\n Migmar Dhakyel\n Willy Rüegg\n Thomas Hartmann').replace('Thomas Largiadèr, SP, Aurel Greter, CVP und Tobias Mani, EVP','Thomas Largiadèr, SP\n Aurel Greter, CVP\n Tobias Mani, EVP').replace('FDP-Fraktion & SVP/BFPW-Fraktion', 'FDP-Fraktion\n SVP/BFPW-Fraktion').replace('Christoph Hirzel, Denise Engel und Thomas Hartmann, SP','Christoph Hirzel\n Denise Engel\n Thomas Hartmann, SP').replace('Gernot Schreiber, EVP und Mitunterzeichnende', 'Gernot Schreiber\n Matthias Haab\n Tobias Mani\n Berti Stocker\n Thomas Hartmann')
            
            splitted_elemente = neueselement.split("\n")
        
            for splitted_element in splitted_elemente:
                cleaned_element = splitted_element.replace("(Mitunterzeichner/in)", "").replace("(Mitunterzeichner)", "").replace("(Unterzeichner/in)", "").replace('(Erstunterzeichner/in)',"").replace('(keine Funktion)',"").replace('(Verfasser/in)',"").replace('(Ersatzurheber/in)',"").replace('(Zweitunterzeichner/in)',"").replace(', SD','').replace(', GLP','').replace(', CVP','').replace(', EVP','').replace(', GP','').replace(', SP','').replace(', SVP','').replace(', FDP','').replace(', Grüne','').replace(', BFPW','').replace(', BDP','').replace(' &',',').replace(' und',',').replace('und Martin Lampert', 'Martin Lampert').replace('Aurèle Greter', 'Aurel Greter').replace('und Aurel Greter', 'Aurel Greter').replace('Signer Karin', 'Karin Signer').replace('Furrer Astrid', 'Astrid Furrer').replace('Thomas Hartmann, 11 mitunterzeichnende Gemeinderäte','Thomas Hartmann').strip()

                if cleaned_element != "":
                    neueliste.append(cleaned_element)
    
    return neueliste

In [9]:
df['Namen-neu'] = df['Namen'].apply(namenliste_reinigen)

In [10]:
df

,Titel,Geschaeftsart,Status,Datum,Namen,Namen-neu
0,Interpellation betreffend Ausbau Bahnhof Wädenswil,Interpellation,Erledigt,2. April 2018,"[ Reiter Ulrich, , Rita Hug (Erstunterzeichner/in)\n Claudia Bühlmann (Mitunterzeichner/in), ]","[Reiter Ulrich, Rita Hug, Claudia Bühlmann]"
1,Postulat betreffend Berücksichtigung der IAO-Kernübereinkommen im städtischen Beschaffungswesen,Postulat,Beantwortet,19. Juni 2008,[Willy Rüegg],[Willy Rüegg]
2,Interpellation betreffend Konzept für den Erhalt der Bienenvölker,Interpellation,Erledigt,17. Dezember 2012,"[ Baer Charlotte, , Jürg Wuhrmann (keine Funktion)\n Ivano Coduri (keine Funktion)\n Erich Schärer (keine Funktion)\n Marcel Bättig (keine Funktion)\n Christoph Lehmann (keine Funktion)\n Adrian Stocker (keine Funktion)\n Pascal Rubin (keine Funktion)\n Christina Zurfluh Fraefel (keine Funktion)\n Hanspeter Andreoli (keine Funktion)\n Sandy Bossert (keine Funktion), , , , , , , , , ]","[Baer Charlotte, Jürg Wuhrmann, Ivano Coduri, Erich Schärer, Marcel Bättig, Christoph Lehmann, Adrian Stocker, Pascal Rubin, Christina Zurfluh Fraefel, Hanspeter Andreoli, Sandy Bossert]"
3,Postulat betreffend Unterstützung zur Senkung von Mietzinsen bei Sozialhilfebeziehenden,Postulat,Abgeschrieben,20. Juni 2017,"[ Reiter Ulrich, , Rita Hug (Erstunterzeichner/in)\n Claudia Bühlmann (Mitunterzeichner/in), ]","[Reiter Ulrich, Rita Hug, Claudia Bühlmann]"
4,Interpellation betreffend die nachhaltige Wasserversorgung in Wädenswil,Interpellation,Beantwortet,15. Juli 2012,[Albert A. Stahel (Verfasser/in)],[Albert A. Stahel]
5,Schriftliche Anfrage betreffend Energieholz für Heizverbund-Anlagen,Anfrage,Beantwortet,16. Oktober 2022,"[ Fuchs Werner, Hauser Urs, Minutella Angelo, Reust Patrick, Roth Hans, Tanner Imfeld Monika, Willi Daniel, , Karin Signer (Mitunterzeichner/in)\n Judith Fürst (Mitunterzeichner/in)\n Edith Höhn (Erstunterzeichner/in)\n Gabi Bachmann (Mitunterzeichner/in), , , ]","[Fuchs Werner, Hauser Urs, Minutella Angelo, Reust Patrick, Roth Hans, Tanner Imfeld Monika, Willi Daniel, Karin Signer, Judith Fürst, Edith Höhn, Gabi Bachmann]"
6,Interpellation betreffend Baurechtsverträge in Wädenswil,Interpellation,Erledigt,22. Juni 2015,"[Thomas Rom (keine Funktion)\n Peter Hildebrand (keine Funktion)\n Beatrice Gmür (Erstunterzeichner/in)\n Hansjörg Schmid (Mitunterzeichner/in)\n Edith Brunner (Mitunterzeichner/in)\n Christian Gross (Mitunterzeichner/in), , , , , ]","[Thomas Rom, Peter Hildebrand, Beatrice Gmür, Hansjörg Schmid, Edith Brunner, Christian Gross]"
7,Interpellation betreffend Parkplätze neue Stadtbibliothek,Interpellation,Beantwortet,30. Januar 2012,"[ Furrer Astrid, , Peter Dolder, Gian-Fadri Gattiker, Thomas Bürge\n Walter Münch (keine Funktion), ]","[Astrid Furrer, Peter Dolder, Gian-Fadri Gattiker, Thomas Bürge, Walter Münch]"
8,Schriftliche Anfrage betreffend Einsatz von kombinierten Abfalleimern mit Aschenbechern,Anfrage,Beantwortet,15. November 2017,"[ Koch Thomas, Schafflützel Roman, Schüpbach Nadia , , Walter Münch (Mitunterzeichner/in)\n Ernst Grand (Mitunterzeichner/in)\n Christian Nufer (Mitunterzeichner/in), , ]","[Koch Thomas, Schafflützel Roman, Schüpbach Nadia, Walter Münch, Ernst Grand, Christian Nufer]"
9,Schriftliche Anfrage betreffend Öffentlichkeitsarbeit zur geplanten Gemeindefusion,Anfrage,Beantwortet,26. März 2016,"[ Roth Hans, , Peter Hildebrand (keine Funktion)\n Beatrice Gmür (Mitunterzeichner/in)\n Hansjörg Schmid (Mitunterzeichner/in)\n Edith Brunner (Erstunterzeichner/in)\n Christian Gross (Mitunterzeichner/in), , , , ]","[Roth Hans, Peter Hildebrand, Beatrice Gmür, Hansjörg Schmid, Edith Brunner, Christian Gross]"


In [11]:
setlocale(LC_TIME, 'de_DE.utf-8')
df['Datum'] = pd.to_datetime(df['Datum'], format='%d. %B %Y')
df.sort_values(by='Datum')

,Titel,Geschaeftsart,Status,Datum,Namen,Namen-neu
359,Postulat betreffend ÖV-Verbesserung ins Reidbach-Quartier (Buslinie 125),Postulat,Beantwortet,2006-04-04,[EVP-Fraktion],[EVP-Fraktion]
364,Schriftliche Anfrage betreffend Missbrauch im Sozialwesen,Kleine Anfrage,Beantwortet,2006-04-28,[Walter Münch (Verfasser/in)],[Walter Münch]
328,Postulat betreffend Einsetzung eines Sozialinspektors,Postulat,None,2006-05-02,[SVP-Fraktion],[SVP-Fraktion]
323,Postulat betreffend Verbesserung der Verkehrssicherheit an der Steinacherstrasse und Bau eines Verkehrskreisels im Bereich Schützenhaus,Postulat,Beantwortet,2006-05-08,[Willy Rüegg],[Willy Rüegg]
259,Postulat betreffend Schaffung von genügend Lehr- und Erwerbsarbeitsstellen für alle Jugendliche in Wädenswil,Postulat,Nicht überwiesen,2006-05-10,[Willy Rüegg],[Willy Rüegg]
252,Schriftliche Anfrage betreffend Kosten für das Logo der Stadt Wädenswil,Kleine Anfrage,Beantwortet,2006-05-11,[ Baer Charlotte],[Baer Charlotte]
287,Schriftliche Anfrage betreffend nicht zugestellter Stimmunterlagen,Kleine Anfrage,Beantwortet,2006-05-21,"[Doris Stüdli, FDP]",[Doris Stüdli]
294,Schriftliche Anfrage betreffend Tiefenhofweiher,Kleine Anfrage,Beantwortet,2006-05-22,[GP-Fraktion],[GP-Fraktion]
226,Motion zur Schaffung einer Ombudsstelle für alle städtischen Abeitlungen,Motion,Beantwortet,2006-06-09,"[Gernot Schreiber, EVP und Mitunterzeichnende]","[Gernot Schreiber, Matthias Haab, Tobias Mani, Berti Stocker, Thomas Hartmann]"
221,Schriftliche Anfrage betreffend Alkohol-Testkäufe,Kleine Anfrage,Beantwortet,2006-06-20,[EVP-Fraktion],[EVP-Fraktion]


In [12]:
df.to_csv('daten_Wädenswil.csv', index=False)